In [1]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append("..")

Now, iText2KG is compatible with all language models supported by LangChain. 

To use iText2KG, you will need both a chat model and an embeddings model. 

For available chat models, refer to the options listed at: https://python.langchain.com/v0.2/docs/integrations/chat/. 
For embedding models, explore the choices at: https://python.langchain.com/v0.2/docs/integrations/text_embedding/. 

This notebook will show you how to run iText2KG using Mistral, Ollama, and OpenAI models. 

**Please ensure that you install the necessary package for each chat model before use.**

# Mistral

For Mistral, please set up your model using the tutorial here: https://python.langchain.com/v0.2/docs/integrations/chat/mistralai/. Similarly, for the embedding model, follow the setup guide here: https://python.langchain.com/v0.2/docs/integrations/text_embedding/mistralai/ .

In [3]:
from langchain_mistralai import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings

mistral_api_key = "##"
mistral_llm_model = ChatMistralAI(
    api_key = mistral_api_key,
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
)


mistral_embeddings_model = MistralAIEmbeddings(
    model="mistral-embed",
    api_key = mistral_api_key
)

c:\Users\yassir.lairgi\Documents\Projects\iText2KG\venv-test\Lib\site-packages\langchain_mistralai\embeddings.py:169: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


# OpenAI

The same applies for OpenAI. 

please setup your model using the tutorial : https://python.langchain.com/v0.2/docs/integrations/chat/openai/
The same for embedding model : https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

openai_api_key = "##"

openai_llm_model = llm = ChatOpenAI(
    api_key = openai_api_key,
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

openai_embeddings_model = OpenAIEmbeddings(
    api_key = openai_api_key ,
    model="text-embedding-3-large",
)

# Ollama

The same applies for Ollama. 

please setup your model using the tutorial : https://python.langchain.com/v0.2/docs/integrations/chat/ollama/
The same for embedding model : https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/

In [5]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

llm = ChatOllama(
    model="llama3",
    temperature=0,
)

embeddings = OllamaEmbeddings(
    model="llama3",
)

# iText2KG

* Use Case: We aim to connect an online job description with a generated CV using Knowledge Graphs. 

* The objective is to assess the candidate's suitability for the job offer. You can utilize different LLM or embedding models for each module of iText2KG. However, it is important to ensure that the dimensions of node and relation embeddings are consistent across models. If the embedding dimensions differ, cosine similarity may struggle to accurately measure vector distances for further matching.

## Document Distiller

### CV

In [6]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(f"../datasets/cvs/CV_Emily_Davis.pdf")
pages = loader.load_and_split()

In [7]:
from itext2kg.documents_distiller import DocumentsDisiller, CV


document_distiller = DocumentsDisiller(llm_model = openai_llm_model)

In [16]:
IE_query = '''
# DIRECTIVES : 
- Act like an experienced information extractor. 
- You have a chunk of a CV.
- If you do not find the right information, keep its place empty.
'''
# we have replaced the curly braces with square brackets to avoid the error in the query
distilled_cv = document_distiller.distill(documents=[page.page_content.replace("{", '[').replace("}", "]") for page in pages], IE_query=IE_query, output_data_structure=CV)

In [17]:
semantic_blocks_cv = [f"{key} - {value}".replace("{", "[").replace("}", "]") for key, value in distilled_doc.items() if value !=[] and value != ""  and value != None]

### Job description

In [18]:
job_offer = """ 
About the Job Offer
THE FICTITIOUS COMPANY

FICTITIOUS COMPANY is a high-end French fashion brand known for its graphic and poetic style, driven by the values of authenticity and transparency upheld by its creator Simon Porte Jacquemus.

Your Role

Craft visual stories that captivate, inform, and inspire. Transform concepts and ideas into visual representations. As a member of the studio, in collaboration with the designers and under the direction of the Creative Designer, you should be able to take written or spoken ideas and convert them into designs that resonate. You need to have a deep understanding of the brand image and DNA, being able to find the style and layout suited to each project.

Your Missions

Translate creative direction into high-quality silhouettes using Photoshop
Work on a wide range of projects to visualize and develop graphic designs that meet each brief
Work independently as well as in collaboration with the studio team to meet deadlines, potentially handling five or more projects simultaneously
Develop color schemes and renderings in Photoshop, categorized by themes, subjects, etc.
Your Profile

Bachelor’s degree (Bac+3/5) in Graphic Design or Art
3 years of experience in similar roles within a luxury brand's studio
Proficiency in Adobe Suite, including Illustrator, InDesign, Photoshop
Excellent communication and presentation skills
Strong organizational and time management skills to meet deadlines in a fast-paced environment
Good understanding of the design process
Freelance contract possibility
"""

In [29]:
job_offer_2 = """ 
About the Job Offer
About The Role

Innovate Design Co. is seeking a talented Visual Designer with expertise in web and interactive design to enhance our visual brand identity. This role is responsible for external digital storytelling and communications design.

You'll be in charge of designing and developing engaging web pages, infographics, social media content, and global digital experiences that effectively convey our cutting-edge technology and products to diverse audiences.

Responsibilities

Translate our core values, product, marketing, and sales objectives into beautifully crafted deliverables
Design compelling, brand-aligned digital and print materials, including websites, social media content, ads, third-party marketplaces, presentations, animations, events, prints, etc.
Develop and maintain visual brand identity guidelines, ensuring brand consistency across all media and multichannel platforms
Communicate Innovate Design Co.'s narrative through conversion and data-driven design
Participate in brainstorming sessions and collaborate with stakeholders to articulate a creative vision that enhances our brand’s visual storytelling
Promote design comprehension and sensibility across the organization, refining work methodologies and design processes to enhance efficiency and effectiveness
Required Qualifications

A Bachelor’s degree (or equivalent) in Graphic Design / Visual Arts - or a self-starter with a strong creative project track record
5-7 years of experience in Graphic Design, including brand design, 360° marketing and communications design, product brand design, 0-to-1 projects, front-end development, etc.
Work experience within well-structured design departments operating in the tech/software space (including leading creative agencies, scale-ups, and mature tech companies)
Proficiency in Figma, Adobe CC, print design best practices, and a solid understanding of web technologies (HTML, CSS, JS)
A robust portfolio demonstrating a variety of design projects, showcasing creativity, originality, consistency, and attention to detail
Perfectly fluent in English, both written and spoken
Results-oriented, resourceful, innovative, intellectually curious, no-ego, proactive
Highly collaborative and able to balance multiple projects and stakeholders
Professional behavior with personal accountability, drive, and work ethics
You may be a good fit if you

Share our excitement for the AI/technology space
Drive projects independently, make judgments based on brand and business goals, seek feedback, collaborate cross-functionally, and leverage others' expertise
Maintain high design standards without perfectionism, understanding tradeoffs in a fast-paced environment
Understand audience feelings, cultures, and challenges through an inquisitive and learning-based approach
What We Offer

The ability to shape the exciting journey of technology and be part of the very early days of one of Europe’s hottest startups
A fun, young, international, and multicultural team — based in Paris, London, and San Francisco
Beautiful office space in the heart of Paris (Canal St Martin)
Competitive salary and benefits package
Opportunities for professional growth and development
"""

In [20]:
from pydantic import BaseModel, Field
from typing import List, Optional

class JobResponsibility(BaseModel):
    description: str = Field(..., description="A specific responsibility in the job role")

class JobQualification(BaseModel):
    skill: str = Field(..., description="A required or preferred skill for the job")

class JobCertification(BaseModel):
    certification: str = Field(..., description="Required or preferred certifications for the job")

class JobOffer(BaseModel):
    job_offer_title: str = Field(..., description="The job title")
    company: str = Field(..., description="The name of the company offering the job")
    location: str = Field(..., description="The job location (can specify if remote/hybrid)")
    job_type: str = Field(..., description="Type of job (e.g., full-time, part-time, contract)")
    responsibilities: List[JobResponsibility] = Field(..., description="List of key responsibilities")
    qualifications: List[JobQualification] = Field(..., description="List of required or preferred qualifications")
    certifications: Optional[List[JobCertification]] = Field(None, description="Required or preferred certifications")
    benefits: Optional[List[str]] = Field(None, description="List of job benefits")
    experience_required: str = Field(..., description="Required years of experience")
    salary_range: Optional[str] = Field(None, description="Salary range for the position")
    apply_url: Optional[str] = Field(None, description="URL to apply for the job")

In [21]:
IE_query = '''
# DIRECTIVES : 
- Act like an experienced information extractor. 
- You have a chunk of a job offer description.
- If you do not find the right information, keep its place empty.
'''
# we have replaced the curly braces with square brackets to avoid the error in the query
distilled_Job_Offer = document_distiller.distill(documents=[job_offer], IE_query=IE_query, output_data_structure=JobOffer)

In [22]:
semantic_blocks_job_offer = [f"{key} - {value}".replace("{", "[").replace("}", "]") for key, value in distilled_Job_Offer.items() if value !=[] and value != ""  and value != None]

In [30]:
distilled_Job_Offer_2 = document_distiller.distill(documents=[job_offer_2], IE_query=IE_query, output_data_structure=JobOffer)
semantic_blocks_job_offer_2 = [f"{key} - {value}".replace("{", "[").replace("}", "]") for key, value in distilled_Job_Offer_2.items() if value !=[] and value != ""  and value != None]

## iText2KG for graph construction

In [84]:
from itext2kg import iText2KG


itext2kg = iText2KG(llm_model = openai_llm_model, embeddings_model = openai_embeddings_model)

We constructed the first graph. As you can see, we have passed all the semantic blocks of the CV to the LLM as one block. You can also pass the job offer as another semantic block, or you can separate the semantic blocks while constructing the graph.

You just need to pay attention to how much information is contained within each block. For example, the block "name - Emily Davis" does not contain much information, which will result in some isolated nodes (even if we reprompt it, it won't converge). It is better to concatenate it with other blocks or pass all the semantic blocks at once as we did here :) .

You should also ensure that you pass a substantial amount of information within each block. If there is a lot of information, you will have a well-formed graph. However, the nodes may contain merged concepts (phrases from paragraphs).

You need to find the optimal amount of information to include (you can experiment by iterating).

In [85]:
global_ent, global_rel = itext2kg.build_graph(sections=[semantic_blocks_cv], ent_threshold=0.6, rel_threshold=0.6)

[INFO] Extracting Entities from the Document 1
{'entities': [{'label': 'Person', 'name': 'Emily Davis'}, {'label': 'Contact Information', 'name': '+1 567 890 1234'}, {'label': 'Contact Information', 'name': 'emilydavis@example.com'}, {'label': 'Social Media', 'name': 'linkedin.com/in/emilydavis'}, {'label': 'Summary', 'name': 'Creative and passionate graphic designer with a keen eye for aesthetics and visual storytelling. Experienced in creating compelling designs for various media, including print and digital platforms.'}, {'label': 'Job Title', 'name': 'Senior Graphic Designer'}, {'label': 'Company', 'name': 'MNO Creative'}, {'label': 'Location', 'name': 'Boston, MA'}, {'label': 'Start Date', 'name': 'June 2017'}, {'label': 'End Date', 'name': 'Present'}, {'label': 'Responsibility', 'name': 'Designed logos, brochures, and social media graphics.'}, {'label': 'Responsibility', 'name': 'Collaborated with clients to understand their vision and deliver high-quality designs.'}, {'label': '

We construct the second graph, noting that we already have existing entities and relationships.

In [86]:
global_ent_, global_rel_ = itext2kg.build_graph(sections=[semantic_blocks_job_offer], existing_global_entities = global_ent, existing_global_relationships = global_rel,  ent_threshold=0.6, rel_threshold=0.6)

[INFO] Extracting Entities from the Document 1
{'entities': [{'label': 'Job Offer Title', 'name': 'Graphic Designer'}, {'label': 'Company', 'name': 'FICTITIOUS COMPANY'}, {'label': 'Job Type', 'name': 'Freelance'}, {'label': 'Responsibility', 'name': 'Translate creative direction into high-quality silhouettes using Photoshop'}, {'label': 'Responsibility', 'name': 'Work on a wide range of projects to visualize and develop graphic designs that meet each brief'}, {'label': 'Responsibility', 'name': 'Work independently as well as in collaboration with the studio team to meet deadlines, potentially handling five or more projects simultaneously'}, {'label': 'Responsibility', 'name': 'Develop color schemes and renderings in Photoshop, categorized by themes, subjects, etc.'}, {'label': 'Qualification', 'name': 'Bachelor’s degree (Bac+3/5) in Graphic Design or Art'}, {'label': 'Qualification', 'name': "3 years of experience in similar roles within a luxury brand's studio"}, {'label': 'Qualifica

Same :)

In [88]:
global_ent__, global_rel__ = itext2kg.build_graph(sections=[semantic_blocks_job_offer_2], existing_global_entities = global_ent_, existing_global_relationships = global_rel_,  ent_threshold=0.6, rel_threshold=0.6)

[INFO] Extracting Entities from the Document 1
{'entities': [{'label': 'Job Offer Title', 'name': 'Visual Designer'}, {'label': 'Company', 'name': 'Innovate Design Co.'}, {'label': 'Location', 'name': 'Paris'}, {'label': 'Location', 'name': 'London'}, {'label': 'Location', 'name': 'San Francisco'}, {'label': 'Responsibility', 'name': 'Translate our core values, product, marketing, and sales objectives into beautifully crafted deliverables'}, {'label': 'Responsibility', 'name': 'Design compelling, brand-aligned digital and print materials, including websites, social media content, ads, third-party marketplaces, presentations, animations, events, prints, etc.'}, {'label': 'Responsibility', 'name': 'Develop and maintain visual brand identity guidelines, ensuring brand consistency across all media and multichannel platforms'}, {'label': 'Responsibility', 'name': "Communicate Innovate Design Co.'s narrative through conversion and data-driven design"}, {'label': 'Responsibility', 'name': 'Pa

# Draw the graph
---

The final section involves visualizing the constructed knowledge graph using GraphIntegrator. The graph database Neo4j is accessed using specified credentials, and the resulting graph is visualized to provide a visual representation of the relationships and entities extracted from the document.

In [89]:
from itext2kg.graph_integration import GraphIntegrator


URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "##"

new_graph = {}
new_graph["nodes"] = global_ent__
new_graph["relationships"] = global_rel__

GraphIntegrator(uri=URI, username=USERNAME, password=PASSWORD).visualize_graph(json_graph=new_graph)